In [10]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)  # reduced display precision on numpy arrays

## pick ap lotting style
print(plt.style.available)  # this shows the available styles
plt.style.use('classic')

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


## Provide data points

each training example has four features: [size, bedrooms, floors, age] and the output is price

The input data is represented by a 2d matrix **X_train** where each row $i$ is a training example x<sup>(i)</sup> 
and each column $j$ is a feature x<sup>(i)</sup><sub>j</sub>

$m$ is the number of examples and $n$ is the number of features, so the 
dimensions of the matrix are $m x n$  

ie it has $m$ rows and $n$ columns

In [6]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

## shape gives you the (m,n) tuple of number of rows (examples) and columns (features)
print(f"X Shape: {X_train.shape}")
print(X_train)
print(f"y Shape: {y_train.shape}")
print(y_train)

X Shape: (3, 4)
[[2104    5    1   45]
 [1416    3    2   40]
 [ 852    2    1   35]]
y Shape: (3,)
[460 232 178]


## Parameters w (vector) b (scalar)
The w parameter is now a vector (we need a value for each of the n features)
b is still a scalar

In [11]:
## optimal initialization values for demonstration of prediction

b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

w_init shape: (4,), b_init type: <class 'float'>


## Model prediction with linear model
The model's price prediction, for a particular input x, is given by the dot product of w and x plus b: 

$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 

In [13]:
## try out a prediction

# get a row from our training data
x_vec = X_train[1,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

prediction = np.dot(x_vec, w_init) + b_init

print(f"predicted cost= {prediction}")

x_vec shape (4,), x_vec value: [1416    3    2   40]
predicted cost= 231.99999836940822


## Compute cost with multiple variables
The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


In contrast to previous labs, $\mathbf{w}$ and $\mathbf{x}^{(i)}$ are vectors rather than scalars supporting multiple features.


In [17]:
## Compute Cost with multiple variables

def compute_cost(X, y, w, b):
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]

    sum = 0.0
    for i in range(m):
      x_vec = X[i,:]
      f_wb_i = np.dot(x_vec, w) + b
      print(f"prediction for example {x_vec}is {f_wb_i}")
      
      sum += (f_wb_i - y[i])**2

    return sum/(2*m)




## call the function to test it
compute_cost(X_train, y_train, w_init, b_init)

prediction for example [2104    5    1   45]is 459.9999976194082
prediction for example [1416    3    2   40]is 231.99999836940822
prediction for example [852   2   1  35]is 177.99999898940825


1.5578904330213735e-12